In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ---------------------------- ----------- 2.9/4.0 MB 16.7 MB/s eta 0:00:01
     ---------------------------------------- 4.0/4.0 MB 17.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ---------------------------------------- 4.0/4.0 MB 34.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
     ---------------------------------------- 4.0/4.0 MB 30.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 41.2 MB/s eta 0:01:06
     ---------------------------------------

In [3]:
pip install -r requirements.txt

  Using cached kagglehub-0.3.4-py3-none-any.whl.metadata (22 kB)
  Using cached yfinance-0.2.50-py2.py3-none-any.whl.metadata (5.5 kB)
  Using cached pandas_ta-0.3.14b.tar.gz (115 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.8.tar.gz (948 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
 

In [6]:
import torch

In [7]:
torch.cuda.is_available()

True

In [8]:
from data_util import get_data_from_kaggle,sentiment_analysis , aggregate_tweet, restructure_date_information, get_static_df, one_label_scale_static_df, scale_stock_data
from constant import Constant
import pandas as pd

c:\Users\luanl\anaconda3\envs\dl_hw3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
constant = Constant()
constant

In [10]:
stock, tweet = get_data_from_kaggle()

In [11]:
tweet

,Date,Tweet,Stock Name,Company Name
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc."
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc."
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc."
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc."
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc."
...,...,...,...,...
80788,2021-10-07 17:11:57+00:00,Some of the fastest growing tech stocks on the...,XPEV,XPeng Inc.
80789,2021-10-04 17:05:59+00:00,"With earnings on the horizon, here is a quick ...",XPEV,XPeng Inc.
80790,2021-10-01 04:43:41+00:00,Our record delivery results are a testimony of...,XPEV,XPeng Inc.
80791,2021-10-01 00:03:32+00:00,"We delivered 10,412 Smart EVs in Sep 2021, rea...",XPEV,XPeng Inc.


In [12]:
stock, tweet = get_data_from_kaggle()

In [13]:
tweet = pd.read_csv('processed.csv')
tweet

,Date,Tweet,Stock Name,Company Name,Positive,Neutral,Negative
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",0.528979,0.351032,0.119989
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",0.012934,0.719147,0.267919
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",0.098909,0.838153,0.062938
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",0.913476,0.075686,0.010838
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",0.971595,0.026244,0.002161
...,...,...,...,...,...,...,...
80788,2021-10-07 17:11:57+00:00,Some of the fastest growing tech stocks on the...,XPEV,XPeng Inc.,0.001424,0.073376,0.925200
80789,2021-10-04 17:05:59+00:00,"With earnings on the horizon, here is a quick ...",XPEV,XPeng Inc.,0.006404,0.497635,0.495962
80790,2021-10-01 04:43:41+00:00,Our record delivery results are a testimony of...,XPEV,XPeng Inc.,0.002887,0.151392,0.845721
80791,2021-10-01 00:03:32+00:00,"We delivered 10,412 Smart EVs in Sep 2021, rea...",XPEV,XPeng Inc.,0.001193,0.061854,0.936952


In [14]:
aggregated_tweet = aggregate_tweet(tweet)
aggregated_tweet

,Stock Name,Date,Positive_Avg,Neutral_Avg,Negative_Avg,Positive_Count,Neutral_Count,Negative_Count
0,AAPL,2021-09-30,0.322527,0.465190,0.212283,3,4,0
1,AAPL,2021-10-01,0.063409,0.574801,0.361790,0,8,3
2,AAPL,2021-10-02,0.053914,0.569441,0.376645,0,2,2
3,AAPL,2021-10-03,0.365725,0.546062,0.088213,0,1,0
4,AAPL,2021-10-04,0.082445,0.740438,0.177116,0,10,0
...,...,...,...,...,...,...,...,...
5905,ZS,2022-09-18,0.002347,0.195517,0.802136,0,0,1
5906,ZS,2022-09-21,0.025520,0.272998,0.701482,0,0,1
5907,ZS,2022-09-27,0.060069,0.497830,0.442101,0,1,0
5908,ZS,2022-09-28,0.129386,0.716569,0.154045,0,1,0


In [15]:
stock['Date'] = pd.to_datetime(stock['Date'] ).dt.date

In [272]:
merged_df = pd.merge(aggregated_tweet, stock, on=['Date', 'Stock Name'], how='right')
merged_df = merged_df.fillna(0)

In [273]:
processed_df = restructure_date_information(merged_df)
processed_df

0


,Stock Name,Positive_Avg,Neutral_Avg,Negative_Avg,Positive_Count,Neutral_Count,Negative_Count,Open,High,Low,Close,Adj Close,Volume,Month,Day,Day of The Week,Week of The Year
0,TSLA,0.203316,0.434594,0.362089,19.0,44.0,27.0,0.257387,0.229337,0.258924,0.244819,0.244819,0.122630,8,29,3,38
1,TSLA,0.174110,0.419891,0.405999,14.0,43.0,37.0,0.253128,0.215174,0.239793,0.244387,0.244387,0.104561,9,0,4,38
2,TSLA,0.198824,0.475948,0.325228,25.0,60.0,34.0,0.282773,0.259596,0.260802,0.254873,0.254873,0.367441,9,3,0,39
3,TSLA,0.218515,0.412092,0.369393,19.0,38.0,31.0,0.263610,0.243211,0.257583,0.253311,0.253311,0.131943,9,4,1,39
4,TSLA,0.164362,0.437867,0.397771,10.0,39.0,29.0,0.249525,0.225147,0.255939,0.256901,0.256901,0.057687,9,5,2,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,XPEV,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012407,0.023899,0.025667,0.037305,0.037305,0.641919,8,22,4,37
6296,XPEV,0.405118,0.451953,0.142929,0.0,1.0,0.0,0.039747,0.045413,0.056417,0.052693,0.052693,0.631516,8,25,0,38
6297,XPEV,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.046640,0.044725,0.043964,0.037305,0.037305,0.451202,8,26,1,38
6298,XPEV,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.011488,0.013096,0.021347,0.028445,0.028445,0.736217,8,27,2,38


In [274]:
processed_df

,Stock Name,Positive_Avg,Neutral_Avg,Negative_Avg,Positive_Count,Neutral_Count,Negative_Count,Open,High,Low,Close,Adj Close,Volume,Month,Day,Day of The Week,Week of The Year
0,TSLA,0.203316,0.434594,0.362089,19.0,44.0,27.0,0.257387,0.229337,0.258924,0.244819,0.244819,0.122630,8,29,3,38
1,TSLA,0.174110,0.419891,0.405999,14.0,43.0,37.0,0.253128,0.215174,0.239793,0.244387,0.244387,0.104561,9,0,4,38
2,TSLA,0.198824,0.475948,0.325228,25.0,60.0,34.0,0.282773,0.259596,0.260802,0.254873,0.254873,0.367441,9,3,0,39
3,TSLA,0.218515,0.412092,0.369393,19.0,38.0,31.0,0.263610,0.243211,0.257583,0.253311,0.253311,0.131943,9,4,1,39
4,TSLA,0.164362,0.437867,0.397771,10.0,39.0,29.0,0.249525,0.225147,0.255939,0.256901,0.256901,0.057687,9,5,2,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,XPEV,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012407,0.023899,0.025667,0.037305,0.037305,0.641919,8,22,4,37
6296,XPEV,0.405118,0.451953,0.142929,0.0,1.0,0.0,0.039747,0.045413,0.056417,0.052693,0.052693,0.631516,8,25,0,38
6297,XPEV,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.046640,0.044725,0.043964,0.037305,0.037305,0.451202,8,26,1,38
6298,XPEV,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.011488,0.013096,0.021347,0.028445,0.028445,0.736217,8,27,2,38


In [275]:
static = get_static_df(processed_df,constant.static_variables)

c:\Users\luanl\Documents\DL\TFT\data_util.py:74: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  static_df.fillna(0, inplace=True)


In [276]:
static[["sector", "country", "beta"]].loc["TSLA"].values

array(['Consumer Cyclical', 'United States', 2.295], dtype=object)

In [277]:
from data_util import get_feature_length

In [278]:
static_cat_feature_length_list, static_cont_feature_length  =   get_feature_length(static, constant.static_variables)
static_cat_feature_length_list, static_cont_feature_length 

([16, 6, 3], 7)

In [279]:
static_df = one_label_scale_static_df(static,constant.static_variables)

In [280]:
from data import TFT_Dataset
x = TFT_Dataset(stock_df= processed_df , 
                static_df=static_df, 
                constant_variable=constant, 
                history_length= 5, 
                prediction_length= 2)

In [281]:
from torch.utils.data import DataLoader

In [282]:
loader = DataLoader(x, batch_size= 10)

In [283]:
count =0
for static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input, prediction in loader:
    break


In [284]:
history_cat_feature_num_list, history_cont_feature_num  =   get_feature_length(processed_df, constant.feature_variables)
static_cat_feature_num_list, static_cont_feature_num  =   get_feature_length(static, constant.static_variables)
future_feature_num,_ =  get_feature_length(processed_df, constant.future_feature)
_, prediction_con  =   get_feature_length(processed_df, constant.prediction_feature)

In [285]:
history_cat_feature_num_list

[12, 31, 5, 52]

In [286]:
future_feature_num

[12, 31, 5, 52]

In [287]:
class TFT_embeding(nn.Module):
    def __init__(self,  static_cont_feature_num = None ,                
                        static_cat_feature_num_list = None , 
                        history_cont_feature_num = None, 
                        history_cat_feature_num_list = None,
                        future_feature_list= None,

                        hidden_size = 64):
        super().__init__()

        self.static_cont = nn.Linear(static_cont_feature_num , hidden_size )
        self.static_cat = nn.ModuleList([
                nn.Embedding(i , hidden_size ) for i in static_cat_feature_num_list ])
        
        self.history_cont = nn.Linear(history_cont_feature_num , hidden_size )
        self.history_cat = nn.ModuleList([
                nn.Embedding(i , hidden_size ) for i in history_cat_feature_num_list ])
        
        self.future_feature = nn.ModuleList([
                nn.Embedding(i , hidden_size ) for i in future_feature_list ])


    def forward(self, 
                static_cont_input, 
                static_cat_input, 
                history_cont_input, 
                history_cat_input, 
                future_input):
        # Static continuous embedding
        static_cont_emb = self.static_cont(static_cont_input)

        # Static categorical embeddings
        static_cat_embs = [emb(static_cat_input[:, i]) 
                        for i, emb in enumerate(self.static_cat)]
        static_cat_emb = torch.cat(static_cat_embs, dim=-1)

        # History continuous embedding
        history_cont_emb = self.history_cont(history_cont_input)

        # History categorical embeddings
        history_cat_embs = [emb(history_cat_input[:, :, i]) 
                                for i, emb in enumerate(self.history_cat)]
        history_cat_emb = torch.cat(history_cat_embs, dim=-1)

        # Future features embedding
        future_emb =  [emb(future_input[:, :, i]) 
                                for i, emb in enumerate(self.future_feature)]

        # Concatenate embeddings
        static_input = torch.cat((static_cont_emb, static_cat_emb), dim=-1)
        history_input = torch.cat((history_cont_emb, history_cat_emb), dim=-1)

        return static_input, history_input, future_emb


In [288]:
emb = TFT_embeding(static_cat_feature_num_list= static_cat_feature_num_list,
             static_cont_feature_num=static_cont_feature_num,
             history_cat_feature_num_list= history_cat_feature_num_list,
             history_cont_feature_num=history_cont_feature_num,
             future_feature_list=future_feature_num,
             hidden_size=64)
emb

TFT_embeding(
  (static_cont): Linear(in_features=7, out_features=64, bias=True)
  (static_cat): ModuleList(
    (0): Embedding(16, 64)
    (1): Embedding(6, 64)
    (2): Embedding(3, 64)
  )
  (history_cont): Linear(in_features=11, out_features=64, bias=True)
  (history_cat): ModuleList(
    (0): Embedding(12, 64)
    (1): Embedding(31, 64)
    (2): Embedding(5, 64)
    (3): Embedding(52, 64)
  )
  (future_feature): ModuleList(
    (0): Embedding(12, 64)
    (1): Embedding(31, 64)
    (2): Embedding(5, 64)
    (3): Embedding(52, 64)
  )
)

In [289]:
future_feature_num

[12, 31, 5, 52]

In [290]:
history_cat_feature_num_list

[12, 31, 5, 52]

In [ ]:

for static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input, prediction in loader:
    emb(static_cont_input, static_cat_input,history_cont_input, history_cat_input, future_input)
    count +=1
